## Data Loading
Let's load the data and take a look into what is in there and how we can use it

In [1]:
#Install packages needed
!pip install -r requirements.txt

In [2]:
## Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

lets load the data from the csv file generated by the scraper

In [3]:
df = pd.read_csv('./get_data/crawl_data.csv')
brands = pd.DataFrame(df.brand.unique()).sort_values(0).reset_index(drop=True)
display(brands)
print('Nº of different engines %s' % len(df.engine.unique()))
df.info()


,0
0,Audi
1,BMW
2,BYD
3,Brilliance
4,Chana
5,Changan
6,Changhe
7,Chery
8,Chevrolet
9,Chrysler


Nº of different engines 296
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2016 entries, 0 to 2015
Data columns (total 16 columns):
brand           2016 non-null object
color           2016 non-null object
engine          1757 non-null object
fuel            1781 non-null object
id              2016 non-null object
image_urls      2016 non-null object
images          2016 non-null object
location        2016 non-null object
mileage         2016 non-null int64
model           2016 non-null object
onlyOwner       1309 non-null object
price           2016 non-null object
steering        2016 non-null object
traction        1365 non-null object
transmission    2016 non-null object
year            2016 non-null int64
dtypes: int64(2), object(14)
memory usage: 252.1+ KB


There is a lot of different type of engines, they are all in different formats (some ar in liters others in C.C),  and some don't even have that, they just put the name.

Because of this there is not an accesible way to make this information reliable so I choose to delete it.

In [4]:
df = df.drop(columns=['engine', 'image_urls', 'images'])
display(df.head())

,brand,color,fuel,id,location,mileage,model,onlyOwner,price,steering,traction,transmission,year
0,Mercedes Benz,Azul,NaN,MCO472077653,El vehículo está en Belmira - Usaquén - Bogotá...,35000,Clase C,NaN,46.950.000,Hidráulica,NaN,Automática,2011
1,Volkswagen,Plateado,Gasolina,MCO474298545,El vehículo está en Bogota - Bogotá D.c.,67700,Jetta,No,31.900.000,Hidráulica,4x2,Mecánica,2014
2,Toyota,Beige,Gasolina,MCO471864641,El vehículo está en Castropol - Medellín - Ant...,97000,Fortuner,NaN,66.900.000,Hidráulica,4x2,Automática,2013
3,Nissan,Blanco,Gasolina,MCO472924395,El vehículo está en Bogota - Bogotá D.c.,135484,Murano,No,35.000.000,Hidráulica,4x4,Automática,2004
4,Renault,Blanco,NaN,MCO471930027,El vehículo está en Los Alpes - Pereira - Risa...,4200,Twizi,Sí,29.990.000,Mecánica,NaN,Automática,2016


next variable to be deal with is the location, this starts with *El vehículo está en* wich needs to be removed, and its also separated by hyphens that also need to be removed

In [5]:
def clean_location(locations, starts_with='El vehículo está en ',):
    location_only_places = locations.str.replace(starts_with, '')
    locations_split = location_only_places.str.split('-')
    location_reverse = locations_split.apply(reversed).apply(list)
    return location_reverse

def get_state_city(location):
    return pd.DataFrame(location.values.tolist())[[0,1]]
    
loc_temp = clean_location(df.location)
df_locations = get_state_city(loc_temp)
df_locations.columns = ['State', 'City']
df = pd.concat([df,df_locations], axis=1)
df = df.drop(columns=['location'])
df = df.set_index('id')

Another element that we must be aware of, is that car manufacturers create diferent cars for diferent *sectors*, back in the day (the beaggining of car manufacturing), a manufacuter had one car and that was it, but as the market grow and the necesities of the people started changing, so did the cars, nowdays a manufacturer tries produces at least on car per **segment**, the reason to consider this is because it does not make any sence to scense to compare a truck with an automovile,that its way we are going to create a new variable from the data we have so far and pulling what we are missing from the internet.

In [13]:
# Import exta libraries
import requests
from bs4 import BeautifulSoup
import urllib
from fuzzywuzzy import fuzz
import unidecode


####
### NOT NEEDING YEARS
#####

# Get the brand and model combination so we can access the car database
brand_model = df[['brand', 'model', 'year']]

car_info = pd.DataFrame(columns=['brand', 'model', 'year', 'doors', 'type'])
car_info.set_index(['brand', 'model', 'year'])

previus_selection = pd.DataFrame(columns=['brand', 'model', 'selection'])

def select_model(option_models, brand, model):
    print('-1 - None of the above')
    for index, option in enumerate(option_models):
        print('{} - {}'.format(index, option['name']))
    selection = int(input('Select a number'))
    if selection == -1:
        return None
    
    return option_models[selection]

def data_exist(brand, model, year):
    try:
        row = car_info.loc[(brand, model, year), :]
        return True
    except KeyError:
        return False

def fetch_data(brand, model, year):
    url_database = 'https://www.cars-data.com/ajax_files/'
    brand_simplified = unidecode.unidecode(brand.lower().replace(' ', '-'))
    url_brand = 'get_groups.php?url={brand}'.format(brand=brand_simplified)
    brand_req = requests.get(url_database + url_brand)
    brand_soup = BeautifulSoup(brand_req.content, "lxml")
    options_models = list(map(
        lambda x: {'value':x['value'],'name': x.text},
        brand_soup.findAll('option')[1:] # The first element is the text "Select model (et.g. Focus)"
    ))
    
    if len(options_models) == 0: # If there is not a available models
        print('No information about the brand {}'.format(brand_simplified))
        return None
    
    similarities_models = list(map(
        lambda x: fuzz.partial_ratio(model, x['name']),
        options_models
    ))
    
    most_likely = np.argsort(similarities_models)[-1]
    model_chosen = options_models[most_likely]['value']
    if similarities_models[most_likely] < 90: # If there is not a specific match it may be because the manufacturer its not right
        model_chosen = select_model(options_models, brand, model)
        model_chosen = model_chosen['value']
        if model_chosen is None:
            print('Nothing got selected')
            
    url_years = 'get_years.php?url={}'.format(model_chosen)
    year_req = requests.get(url_database + url_years)
    year_soup = BeautifulSoup(year_req.content, 'lxml')
    options_year = list(map(
        lambda x: {'value': x['value'], 'name': x.text},
        year_soup.findAll('option')
    ))
    
    new_df = pd.DataFrame()
    print('YEARS OPTIONS {}'.format(options_year))
    for year_option in options_year:
        year_chosen_value = year_option['value']
        year_chosen_text = year_option['name']
        car_uri = year_chosen_value.split('|')[1]
        car_url = 'https://www.cars-data.com/en/{}'.format(car_uri)
        
        car_req = requests.get(car_url)
        car_soup = BeautifulSoup(car_req.content, 'lxml')
        section_p = car_soup.find('section', {'class': 'title'}).p.text
        car_description = section_p.split(',')[1]
        car_spec = car_description.split()[:2]
        new_df = new_df.append(pd.DataFrame({'brand': [brand],
                                'year': [year_chosen_text],
                                'model': [model],
                                'doors':  [car_spec[0]],
                                'type': [car_spec[1]]})
                              )
    return new_df
    
def get_data(brand, model, year):
    if data_exist(brand, model, year):
        return
    new_data = fetch_data(brand, model, year)
    return new_data

for index, row in df.iterrows():
    brand, model, year = row[['brand', 'model', 'year']]
    print(row)
    car_info = car_info.append(get_data(brand, model, year))
    display(car_info)
    
    

Empty DataFrame
Columns: [brand, model, year, doors, type]
Index: []
brand           Mercedes Benz
color                    Azul
fuel                      NaN
mileage                 35000
model                 Clase C
onlyOwner                 NaN
price              46.950.000
steering           Hidráulica
traction                  NaN
transmission       Automática
year                     2011
State             Bogotá D.c.
City                 Usaquén 
Name: MCO472077653, dtype: object
-1 - None of the above
0 - 190-serie Sedan
1 - 200-serie Sedan
2 - 200-serie Cabrio Convertible
3 - 200-serie Combi Wagon
4 - 200-serie Coupe Coupe
5 - A-class Hatchback
6 - A-class Mpv
7 - A-class Coupe Mpv
8 - A-class Lang Mpv
9 - AMG GT Coupe
10 - AMG GT Roadster Convertible
11 - B-class Mpv
12 - B-class Electric Drive Mpv
13 - C-class Sedan
14 - C-class Cabriolet Convertible
15 - C-class Combi Wagon
16 - C-class Coupe Coupe
17 - C-class Estate Wagon
18 - C-class Sportcoupe Coupe
19 - C-class Sports

,brand,doors,model,type,year
0,Mercedes Benz,4-door,Clase C,sedan,1993
0,Mercedes Benz,4-door,Clase C,sedan,1995
0,Mercedes Benz,4-door,Clase C,sedan,1997
0,Mercedes Benz,4-door,Clase C,sedan,2000
0,Mercedes Benz,4-door,Clase C,sedan,2004
0,Mercedes Benz,4-door,Clase C,sedan,2007
0,Mercedes Benz,4-door,Clase C,sedan,2011
0,Mercedes Benz,4-door,Clase C,sedan,2014


brand             Volkswagen
color               Plateado
fuel                Gasolina
mileage                67700
model                  Jetta
onlyOwner                 No
price             31.900.000
steering          Hidráulica
traction                 4x2
transmission        Mecánica
year                    2014
State            Bogotá D.c.
City                 Bogota 
Name: MCO474298545, dtype: object
YEARS OPTIONS [{'value': '1980|volkswagen-jetta-1980/2811', 'name': '1980'}, {'value': '1981|volkswagen-jetta-1981/2812', 'name': '1981'}, {'value': '1984|volkswagen-jetta-1984/2813', 'name': '1984'}, {'value': '1986|volkswagen-jetta-1986/2814', 'name': '1986'}, {'value': '1980|volkswagen-jetta-1980/2815', 'name': '1980'}, {'value': '1981|volkswagen-jetta-1981/2816', 'name': '1981'}, {'value': '1984|volkswagen-jetta-1984/2817', 'name': '1984'}, {'value': '1986|volkswagen-jetta-1986/2818', 'name': '1986'}, {'value': '2005|volkswagen-jetta-2005/2819', 'name': '2005'}, {'value': '2011|

,brand,doors,model,type,year
0,Mercedes Benz,4-door,Clase C,sedan,1993
0,Mercedes Benz,4-door,Clase C,sedan,1995
0,Mercedes Benz,4-door,Clase C,sedan,1997
0,Mercedes Benz,4-door,Clase C,sedan,2000
0,Mercedes Benz,4-door,Clase C,sedan,2004
0,Mercedes Benz,4-door,Clase C,sedan,2007
0,Mercedes Benz,4-door,Clase C,sedan,2011
0,Mercedes Benz,4-door,Clase C,sedan,2014
0,Volkswagen,2-door,Jetta,sedan,1980
0,Volkswagen,2-door,Jetta,sedan,1981


brand               Toyota
color                Beige
fuel              Gasolina
mileage              97000
model             Fortuner
onlyOwner              NaN
price           66.900.000
steering        Hidráulica
traction               4x2
transmission    Automática
year                  2013
State            Antioquia
City             Medellín 
Name: MCO471864641, dtype: object
-1 - None of the above
0 - 4Runner Suv
1 - Auris Hatchback
2 - Auris Touring Sports Wagon
3 - Avensis Hatchback
4 - Avensis Sedan
5 - Avensis Touring Sports Wagon
6 - Avensis Verso Mpv
7 - Avensis Wagon Wagon
8 - Aygo Hatchback
9 - C-HR Suv
10 - Camry Hatchback
11 - Camry Sedan
12 - Camry Customwagon Wagon
13 - Camry Stationwagon Wagon
14 - Carina Hatchback
15 - Carina Sedan
16 - Carina Combi Wagon
17 - Carina E Hatchback
18 - Carina E Sedan
19 - Carina E Stationwagon Wagon
20 - Carina II Hatchback
21 - Carina II Sedan
22 - Carina II Stationwagon Wagon
23 - Celica Coupe
24 - Celica Convertible Convertible
25

...

,brand,doors,model,type,year
0,Mercedes Benz,4-door,Clase C,sedan,1993
0,Mercedes Benz,4-door,Clase C,sedan,1995
0,Mercedes Benz,4-door,Clase C,sedan,1997
0,Mercedes Benz,4-door,Clase C,sedan,2000
0,Mercedes Benz,4-door,Clase C,sedan,2004
0,Mercedes Benz,4-door,Clase C,sedan,2007
0,Mercedes Benz,4-door,Clase C,sedan,2011
0,Mercedes Benz,4-door,Clase C,sedan,2014
0,Volkswagen,2-door,Jetta,sedan,1980
0,Volkswagen,2-door,Jetta,sedan,1981


brand                   Audi
color                   Azul
fuel                Gasolina
mileage                50000
model                     A4
onlyOwner                NaN
price             60.500.000
steering          Hidráulica
traction                 4x2
transmission      Automática
year                    2012
State            Bogotá D.c.
City                   Suba 
Name: MCO468729409, dtype: object
YEARS OPTIONS [{'value': '2002|audi-a4-cabriolet-2002/142', 'name': '2002'}, {'value': '2005|audi-a4-cabriolet-2005/143', 'name': '2005'}]


,brand,doors,model,type,year
0,Mercedes Benz,4-door,Clase C,sedan,1993
0,Mercedes Benz,4-door,Clase C,sedan,1995
0,Mercedes Benz,4-door,Clase C,sedan,1997
0,Mercedes Benz,4-door,Clase C,sedan,2000
0,Mercedes Benz,4-door,Clase C,sedan,2004
0,Mercedes Benz,4-door,Clase C,sedan,2007
0,Mercedes Benz,4-door,Clase C,sedan,2011
0,Mercedes Benz,4-door,Clase C,sedan,2014
0,Volkswagen,2-door,Jetta,sedan,1980
0,Volkswagen,2-door,Jetta,sedan,1981


brand           Mercedes Benz
color                  Blanco
fuel                 Gasolina
mileage                 76800
model            Clase ML 350
onlyOwner                  No
price              63.900.000
steering           Hidráulica
traction                  4x4
transmission       Automática
year                     2007
State             Bogotá D.c.
City               Chapinero 
Name: MCO470975038, dtype: object
-1 - None of the above
0 - 190-serie Sedan
1 - 200-serie Sedan
2 - 200-serie Cabrio Convertible
3 - 200-serie Combi Wagon
4 - 200-serie Coupe Coupe
5 - A-class Hatchback
6 - A-class Mpv
7 - A-class Coupe Mpv
8 - A-class Lang Mpv
9 - AMG GT Coupe
10 - AMG GT Roadster Convertible
11 - B-class Mpv
12 - B-class Electric Drive Mpv
13 - C-class Sedan
14 - C-class Cabriolet Convertible
15 - C-class Combi Wagon
16 - C-class Coupe Coupe
17 - C-class Estate Wagon
18 - C-class Sportcoupe Coupe
19 - C-class Sports Coupe Coupe
20 - Citan Tourer Mpv
21 - CL-class Coupe
22 - CLA-class

KeyboardInterrupt: 